# Финальное задание

1) Считываем данные из файла features.csv.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
import time

features = pd.read_csv('../../data/final/features.csv', index_col='match_id')

2) Удаляем признаки, связанные с итогом матча.

In [2]:
features_from_future = ["duration", "radiant_win", "tower_status_radiant",
                        "tower_status_dire", "barracks_status_dire", 
                        "barracks_status_radiant"]
X = features.drop(features_from_future, axis=1)

3) Вывод признаков, в которых отсутствуют значения. Часть из них могла быть пропущена из-за того, что эти действия редко отслеживаются тщательно (dire_first_ward_time и radiant_first_ward_time), либо в это же время в матче происходили куда более влияющие на исход игры события - сражения между игроками и т.д.

In [3]:
count = X.count()
print(count[count < len(X)].index.values)

['first_blood_time' 'first_blood_team' 'first_blood_player1'
 'first_blood_player2' 'radiant_bottle_time' 'radiant_courier_time'
 'radiant_flying_courier_time' 'radiant_first_ward_time' 'dire_bottle_time'
 'dire_courier_time' 'dire_flying_courier_time' 'dire_first_ward_time']


4) Заполняем пропуски нулями.

In [4]:
X = X.fillna(0)

5) Столбец, содержащий значение целевой переменной:

In [5]:
Y = features['radiant_win']
print('radiant_win')

radiant_win


## Градентный бустинг. 

Для замера времени используется magic-функция IPython - %%time. 

In [6]:
kfold = KFold(shuffle=True, n_splits=5, random_state=241)
gb10 = GradientBoostingClassifier(n_estimators=10)
gb20 = GradientBoostingClassifier(n_estimators=20)
gb30 = GradientBoostingClassifier(n_estimators=30)
gb40 = GradientBoostingClassifier(n_estimators=40)

In [7]:
gb10score = cross_val_score(gb10, X, y=Y, scoring='roc_auc', cv=kfold)
print(np.mean(gb10score))

0.664387720635


In [8]:
gb20score = cross_val_score(gb20, X, y=Y, scoring='roc_auc', cv=kfold)
print(np.mean(gb20score))

0.682853573534


In [9]:
%%time
gb30score = cross_val_score(gb30, X, y=Y, scoring='roc_auc', cv=kfold)
print(np.mean(gb30score))

0.689496206059
CPU times: user 1min 2s, sys: 144 ms, total: 1min 2s
Wall time: 1min 2s


In [10]:
gb40score = cross_val_score(gb40, X, y=Y, scoring='roc_auc', cv=kfold)
print(np.mean(gb40score))

0.694131121473


Кроссвалидация градиентного бустинга с 30 деревьями заняла 1 минуту 2 секунды. Полученное качество - 0.689 - достаточно средний показатель. Увеличение числа деревьев может улучшить качество (например, на 40 деревьях качество уже 0.694), однако рост качества будет замедляться и, кроме того, возможно возникновение переобучения. Чтобы ускорить обучение, можно распараллелить кроссвалидацию (самый очевидный путь, однако прирост будет не очень большим), либо уменьшить глубину деревьев, но тогда возможно падение качества.

## Логистическая регрессия.

Подбор параметра регуляризации C в пределах от 0.0001 до 10.

In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [47]:
grid1 = GridSearchCV(LogisticRegression(C=1.0),
                   {'C': np.linspace(0.00001, 1, num=10)}, 
                    cv=kfold, scoring='roc_auc')
grid1.fit(X_scaled, Y);
best_c = grid1.best_params_['C']
print(grid1.best_score_)
print(best_c)

0.716309687427
0.11112


Значение ROC-AUC метрики получилось равным 0.653 при коэффициенте регуляризации C = 0.11.

In [46]:
%%time 
cls = LogisticRegression(C=best_c)
print(np.mean(cross_val_score(cls, X_scaled, y=Y, scoring='roc_auc', cv=kfold)))

0.7163069205
CPU times: user 13.7 s, sys: 1.2 s, total: 14.9 s
Wall time: 13.7 s


Уберем из выборки категориальные признаки.

In [14]:
cat_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_wc = scaler.fit_transform(X.drop(cat_features, axis=1))

In [15]:
grid2 = GridSearchCV(LogisticRegression(C=1.0),
                   {'C': np.linspace(0.00001, 1, num=10)}, 
                    cv=kfold, scoring='roc_auc')
grid2.fit(X_wc, Y);
best_c = grid2.best_params_['C']
print(grid2.best_score_)
print(best_c)

0.716373371391
0.11112


In [34]:
def bow_heroes(X, hero_cols):
    N = len(X[cat_features[1:]].stack().unique())
    max_hero_id = max(X[cat_features[1:]].max(axis=0))
    diff = max_hero_id - N
    X_pick = np.zeros((X.shape[0], N))

    for i, match_id in enumerate(X.index):
        for p in range(5):
            X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1-diff] = 1
            X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1-diff] = -1
    return X_pick

In [35]:
X_pick = bow_heroes(X, cat_features)
X_final = np.hstack((X_wc, X_pick))

In [ ]:
grid3 = GridSearchCV(LogisticRegression(C=1.0),
                   {'C': np.linspace(0.00001, 1, num=10)}, 
                    cv=kfold, scoring='roc_auc')
grid3.fit(X_final, Y);

In [19]:
best_c = grid3.best_params_['C']
print(grid3.best_score_)
print(best_c)

0.749907823455
0.11112


In [43]:
test = pd.read_csv('../../data/final/features_test.csv', index_col='match_id')
test = test.fillna(0)
best_model = grid3.best_estimator_
test_wc = scaler.fit_transform(test.drop(cat_features, axis=1))
test_cat = bow_heroes(test, cat_features)
test = np.hstack((test_wc, test_cat))
predict = best_model.predict_proba(test)
print(np.max(predict))
print(np.min(predict))

0.997536638423
0.00246336157663
